#**Libraries / path definition / various functions**

In [ ]:
!pip install torchmetrics

Libraries

In [ ]:
import os
import sys
import random
import numpy as np
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import datasets, transforms
from torchvision.datasets import VisionDataset
import torchvision.transforms.functional as Fv
import torchmetrics
from torchmetrics import JaccardIndex
from transformers import get_scheduler

Some path to adapt accordingly

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
project_dir = '/content/gdrive/MyDrive/Schism/'
sys.path.append(os.path.join(project_dir, "code/DinoV2/"))

# Define the project directory path
#project_dir = '/content/gdrive/MyDrive/'

# Add the DinoV2 code directory to the system path for module imports
sys.path.append(os.path.join(project_dir, "code/DinoV2/"))

# Define the runs directory path within the project directory
runs_directory = os.path.join(project_dir, 'runs')

# Define the data directory path within the project directory
data_directory = os.path.join(project_dir, 'data')

Some functions


In [ ]:
class UnetSegmentor(nn.Module):
    def __init__(self, n_blocks=4, n_filter=32, num_classes=1, p=0.5):
        super(UnetSegmentor, self).__init__()
        self.n_blocks = n_blocks
        self.p = p
        self.input_conv = nn.Conv2d(in_channels=3, out_channels=n_filter, kernel_size=3, padding=1)
        self.encoder_convs = nn.ModuleList([self._create_encoder_conv_block(channels=n_filter*2**i, kernel_size=3) for i in range(0,n_blocks-1)])
        self.mid_conv = self._create_encoder_conv_block(channels=n_filter*2**(n_blocks-1),  kernel_size=3)
        self.decoder_deconvs = nn.ModuleList([nn.ConvTranspose2d(in_channels=n_filter*2**(i+1), out_channels=n_filter*2**i, kernel_size=3, stride=2, padding=1, output_padding=1) for i in reversed(range(n_blocks))])
        self.decoder_convs = nn.ModuleList([self._create_decoder_conv_block(channels=n_filter*2**i, kernel_size=3) for i in reversed(range(n_blocks))])
        self.seg_conv = nn.Conv2d(in_channels=n_filter, out_channels=num_classes, kernel_size=3, padding=1)

    def _create_encoder_conv_block(self, channels, kernel_size):
            return nn.Sequential(
                nn.Conv2d(channels, channels*2, kernel_size=kernel_size, padding=1),
                nn.BatchNorm2d(channels*2),
                nn.ReLU(),
                nn.Conv2d(channels*2, channels*2, kernel_size=kernel_size, padding=1),
                nn.BatchNorm2d(channels*2),
                nn.ReLU(),
            )

    def _create_decoder_conv_block(self, channels, kernel_size):
            return nn.Sequential(
                nn.Conv2d(channels, channels, kernel_size=kernel_size, padding=1),
                nn.BatchNorm2d(channels),
                nn.ReLU(),
                nn.Conv2d(channels, channels, kernel_size=kernel_size, padding=1),
                nn.BatchNorm2d(channels),
                nn.ReLU(),
            )

    def forward(self, x):
        feature_list = []
        x = self.input_conv(x)
        feature_list.append(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = F.dropout(x, p=self.p)
        for i in range(self.n_blocks-1):
            x = self.encoder_convs[i](x)
            feature_list.append(x)
            x = F.max_pool2d(x, kernel_size=2)
            x = F.dropout(x, p=self.p)

        x = self.mid_conv(x)

        for i in range(self.n_blocks):
            x = self.decoder_deconvs[i](x)
            x = F.dropout(x, p=self.p)
            x = self.decoder_convs[i](x+feature_list[::-1][i])

        return self.seg_conv(x)

def load_segmentation_data(data_dir, device, img_res, num_samples, train_rocks, test_rocks, num_classes=1, val_split=0.8, batch_size=2):
    # Initialize the training dataset with specified parameters
    train_dataset = EfficientSegmentationDataset(
        data_dir, train_rocks, num_samples=num_samples, num_classes=num_classes, img_res=img_res
    )

    # Initialize the testing dataset with specified parameters (num_samples set to 500 by default)
    test_dataset = EfficientSegmentationDataset(
        data_dir, test_rocks, num_samples=500, num_classes=num_classes, img_res=img_res
    )

    pin_memory_device = 'cuda'

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True, pin_memory_device=pin_memory_device)
    val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True, pin_memory_device=pin_memory_device)

    return {'train': train_loader, 'val': val_loader, 'num_classes': num_classes}

class EfficientSegmentationDataset(VisionDataset):
    def __init__(self, data_dir, rock_names, num_classes=3, num_samples=None, crop_size = (224,224), p=0.5, img_res=224, save_dir=None):
        super().__init__(data_dir, transforms=None)
        print("Loading data ...")
        # The stats have been computed on our side. These values represents the mean and stddev pour each rock dataset
        self.data_stats = {
            "sample1" : [
                np.array([123.07921846875976]*3)/255.0, np.array([84.04993142526148]*3)/255.0
            ],

            "sample2" : [
                np.array([117.92807255795907]*3)/255.0, np.array([80.61479412614699]*3)/255.0
            ],

            "sample3" : [
                np.array([119.7933619436969]*3)/255.0, np.array([80.18348841827216]*3)/255.0
            ],

        }
        self.img_data = [np.lib.format.open_memmap(data_dir+f"/{rock}_img.npy", dtype=np.uint8, mode='r') for rock in rock_names]
        self.mask_data = [np.lib.format.open_memmap(data_dir+f"/{rock}_mask.npy", dtype=np.uint8, mode='r') for rock in rock_names]
        self.rock_names = rock_names
        self.crop_size=crop_size
        self.p=p
        self.OFFSET = 128
        self.IMG_RES = img_res
        self.save_dir = save_dir
        self.num_classes = num_classes
        self.inference_mode = False

        if num_samples is None:
          self.num_samples = len(self.img_data[0])
        else:
          self.num_samples = num_samples

        self.num_datasets = len(self.img_data)

    def infer(self):
        self.inference_mode = True

    def train(self):
        self.inference_mode = False

    def get_random_crop_params(self):
        h, w = (1014, 976)
        th, tw = self.crop_size

        if h < th or w < tw:
            raise ValueError(f"Required crop size {(th, tw)} is larger than input image size {(h, w)}")

        if w == tw and h == th:
            return 0, 0, h, w

        i = torch.randint(0, h - th + 1, size=(1,)).item()
        j = torch.randint(0, w - tw + 1, size=(1,)).item()

        return i, j, th, tw


    def _weights_calc(self, mask, temperature=50.0):
        # Calculate the class frequencies
        values_to_count = [85, 170, 255]
        counts = np.bincount(mask.ravel())[values_to_count]
        class_ratio = counts / np.sum(counts)
        u_weights = 1/class_ratio
        weights = np.nan_to_num(u_weights, posinf=-np.inf)
        weights = F.softmax(torch.from_numpy(weights).float()/temperature, dim=-1)
        if torch.any(torch.isnan(weights)):
            print(weights)
            print(class_ratio)
            print(u_weights)
            raise
        return weights

    def non_local_means_filter(self, image, h=10, templateWindowSize=7, searchWindowSize=21):
        return cv2.fastNlMeansDenoising(image, None, h, templateWindowSize, searchWindowSize)

    def __getitem__(self, idx):

        try:
            i, j, h, w = self.get_random_crop_params()
            idx_x = i
            idx_y = j
            if idx_x < self.OFFSET:
                idx_x += self.OFFSET
            if idx_y < self.OFFSET:
                idx_y += self.OFFSET

            dataset_index = idx % self.num_datasets
            data_idx = (idx // self.num_datasets)

            img = self.img_data[dataset_index][data_idx, idx_x:h+idx_x, idx_y:w+idx_y, :].copy()
            mask = self.mask_data[dataset_index][data_idx, idx_x:h+idx_x, idx_y:w+idx_y].copy()

            if img is None or mask is None:
                raise ValueError("Image or mask is None.")

            weights = self._weights_calc(mask)

            img = torch.from_numpy(img.transpose((2, 0, 1))).contiguous() / 255.0
            mask = torch.from_numpy(mask).contiguous() / 255.0

            img = F.interpolate(input=img.unsqueeze(0), size=(self.IMG_RES, self.IMG_RES), mode="bicubic", align_corners=False).squeeze()
            mask = F.interpolate(input=mask.unsqueeze(0).unsqueeze(0), size=(self.IMG_RES, self.IMG_RES), mode="nearest").squeeze()

            if torch.rand(1) < self.p and not self.inference_mode:
                img = torchvision.transforms.functional.hflip(img)
                mask = torchvision.transforms.functional.hflip(mask)

            if torch.rand(1) < self.p and not self.inference_mode:
                img = torchvision.transforms.functional.vflip(img)
                mask = torchvision.transforms.functional.vflip(mask)

            img_unalt = img

            contrast = 0.5 * torch.rand(1)
            img = Fv.adjust_contrast(img, 0.75 + contrast)

            if not self.inference_mode:
                brightness = random.uniform(0.8, 1.2)
                contrast = random.uniform(0.8, 1.2)
                img = torchvision.transforms.functional.adjust_brightness(img, brightness)
                img = torchvision.transforms.functional.adjust_contrast(img, contrast)

                gamma = random.uniform(0.5, 1.5)
                img = torchvision.transforms.functional.adjust_gamma(img, gamma)

            m = self.data_stats[self.rock_names[dataset_index]][0]
            s = self.data_stats[self.rock_names[dataset_index]][1]

            if self.num_classes > 2:
                mask = (mask * self.num_classes).long() - 1

            img = torchvision.transforms.functional.normalize(img, m, s).float()
            return img, mask, weights, img_unalt

        except Exception as e:
            print(f"Error processing index {idx}: {e}")
            # Return a valid default value (e.g., an empty tensor) to avoid returning None
            return torch.zeros(3, self.IMG_RES, self.IMG_RES), torch.zeros(self.IMG_RES, self.IMG_RES), torch.zeros(self.IMG_RES, self.IMG_RES), torch.zeros(3, self.IMG_RES, self.IMG_RES)

    def __len__(self):
        return self.num_datasets * self.num_samples

Variable settings

In [ ]:
# Define the name of the folder containining the datasets. All data are in npy format.
# The expected data directory structure is as follows:
# Datasets
# |_Sample1
# |  |_img.npy
# |  |_mask.npy
# |_Sample2
# |  |_img.npy
# |  |_mask.npy
# ...
dataset_name = "Alhammadi"

# Set the device to 'cuda' for GPU usage
device = 'cuda'

# Define the training and testing rock samples
train_rocks = ['sample1', 'sample2']
test_rocks = ["sample3"]

# Set the batch size for data loading
batch_size = 15

# Set the number of epochs for training
epoch_nb = 20

# Specify the size of the model (small or large)
size_network = "small"

# Set the image resolution.
img_res = 560

# Set the number of samples to be used for training.
# This number represents the amount of 2D slice that will be randomly selected per dataset (in our case 500*2)
num_samples = 500

# Using the datasets that carry 3 classes
num_classes = 3

# Set the learning rate
lr=1e-4

# set the weight decay
weight_decay=5e-4

# Get the number of datasets based on the length of training rocks
num_dataset = len(train_rocks)

# Folder used to save the weights/checkpoints after training and/or read the weights for inference
save_directory = runs_directory +'/UNet_'+ f"{size_network}_{num_samples}_{img_res}/"

# Data loading
data_npy = os.path.join(project_dir, "npy_data", dataset_name)
dataloaders = load_segmentation_data(data_dir=data_npy,
                                     device=device,
                                     img_res=img_res,
                                     num_samples=num_samples,
                                     train_rocks=train_rocks,
                                     test_rocks=test_rocks,
                                     num_classes=num_classes,
                                     batch_size=batch_size)

# **Training**

Just run this to get the magic done

In [ ]:
if not os.path.exists(save_directory):
		os.makedirs(save_directory)

print(save_directory)

# Get the number of datasets based on the length of training rocks
num_dataset = len(train_rocks)

# Define the directory to save the results
save_directory = runs_directory +'/UNet_'+ f"{size_network}_{num_samples}_{img_res}/"
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

print(save_directory)

def plot_learning_curves(loss_dict, jaccard_dict, MODEL_FOLDER):
    """
    Plots the learning curves for loss and IoU over epochs.

    Args:
        loss_dict (dict): Dictionary containing loss values for training and validation.
        jaccard_dict (dict): Dictionary containing IoU values for training and validation.
        MODEL_FOLDER (str): Directory to save the plot.
    """
    # Extract epochs and corresponding loss and IoU values
    epochs = list(loss_dict['train'].keys())
    train_loss_values = [loss_dict['train'][epoch] for epoch in epochs]
    val_loss_values = [loss_dict['val'][epoch] for epoch in epochs]
    train_jaccard_values = [jaccard_dict['train']['mean_iou'][epoch - 1] for epoch in epochs]
    val_jaccard_values = [jaccard_dict['val']['mean_iou'][epoch - 1] for epoch in epochs]

    # Create subplots for loss and IoU
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    ax0 = axes[0]
    ax2 = axes[1]

    # Plot loss values
    ax0.plot(epochs, train_loss_values, 'b-', label='train')
    ax0.plot(epochs, val_loss_values, 'r-', label='val')
    ax0.set_title('Loss')
    ax0.set_xlabel('Epochs')
    ax0.set_ylabel('Loss')

    # Plot IoU values
    ax2.plot(epochs, train_jaccard_values, 'b-', label='train')
    ax2.plot(epochs, val_jaccard_values, 'r-', label='val')
    ax2.set_title('IoU')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('%')
    ax2.set_ylim(0.1, 1.0)
    ax2.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(save_directory, 'learning_curves.png'), dpi=300)

def save_experiment_params(num_samples_train, batch_size, num_dataset, EPOCH_NB, lr, weight_decay, model):
    """
    Saves the experiment parameters and model state dictionary.

    Args:
        num_samples_train (int): Number of training samples.
        batch_size (int): Batch size.
        num_dataset (int): Number of datasets.
        EPOCH_NB (int): Number of epochs.
        lr (float): Learning rate.
        weight_decay (float): Weight decay.
        model (torch.nn.Module): Trained model.
    """
    # Create folder name based on current date and time
    current_time = datetime.now().strftime("%d-%m-%Y-%H-%M")
    folder_name = f'UNET-{current_time}'

    # Create the directory if it doesn't exist
    folder_path = os.path.join(save_directory, folder_name)
    os.makedirs(folder_path, exist_ok=True)

    # Save the experiment parameters to a text file
    params_file_path = os.path.join(folder_path, 'experiment_params.txt')
    with open(params_file_path, 'w') as f:
        f.write(f'num_samples_train={num_samples_train}\n')
        f.write(f'batch_size={batch_size}\n')
        f.write(f'num_dataset={num_dataset}\n')
        f.write(f'EPOCH_NB={EPOCH_NB}\n')
        f.write(f'lr={lr}\n')
        f.write(f'weight_decay={weight_decay}\n')

    # Save the model state dict
    model_file_path = os.path.join(folder_path, 'model_state_dict.pth')
    torch.save(model.state_dict(), model_file_path)

def save_dicts(loss_dict, jaccard_dict, file_path):
    """
    Saves the loss and IoU dictionaries as PyTorch tensors.

    Args:
        loss_dict (dict): Dictionary containing loss values for training and validation.
        jaccard_dict (dict): Dictionary containing IoU values for training and validation.
        file_path (str): Directory to save the tensors.
    """
    # Convert dictionaries to PyTorch tensors
    loss_tensor = {
        'train': torch.tensor(list(loss_dict['train'].values())),
        'val': torch.tensor(list(loss_dict['val'].values()))
    }

    jaccard_tensor = {
        'train': torch.tensor(jaccard_dict['train']['mean_iou']),
        'val': torch.tensor(jaccard_dict['val']['mean_iou'])
    }

    # Save tensors to files
    torch.save(loss_tensor, os.path.join(file_path, 'loss_tensor.pt'))
    torch.save(jaccard_tensor, os.path.join(file_path, 'iou_tensor.pt'))

def train_segmentation_model(model, dataloaders, criterion, optimizer, scheduler, device, epoch_nb=10):
    """
    Trains the segmentation model.

    Args:
        model (torch.nn.Module): The segmentation model.
        dataloaders (dict): Dictionary containing training and validation dataloaders.
        criterion (torch.nn.Module): Loss function.
        optimizer (torch.optim.Optimizer): Optimizer.
        scheduler (torch.optim.lr_scheduler._LRScheduler): Learning rate scheduler.
        device (str): Device to run the training on ('cuda' or 'cpu').
        epoch_nb (int): Number of epochs to train.

    Returns:
        torch.nn.Module: Trained model.
    """
    if device == "cuda":
        scaler = torch.cuda.amp.GradScaler()
        import torch.backends.cudnn as cudnn
        cudnn.benchmark = True

    jaccard = torchmetrics.classification.MulticlassJaccardIndex(num_classes=3, ignore_index=-1).to(device)

    print('New model')
    epoch_init = 1
    loss_dict = {}
    metrics_dict = {}
    monitored_metrics = ["mean_iou"]
    for phase in ['train', 'val']:
        loss_dict[phase] = {}
        metrics_dict[phase] = {}
        for m in monitored_metrics:
            metrics_dict[phase][m] = []

    best_train_iou = 0
    best_val_iou = 0
    best_val_loss = 100000
    for epoch in range(epoch_init, epoch_init+epoch_nb):
        print('\n')
        print('-' * 18)
        print('--- Epoch {}/{} ---'.format(epoch, epoch_init+epoch_nb-1))

        for phase in ['train', 'val']:
            if phase == 'train':
                is_training = True
                model.train()
            else:
                model.eval()
                is_training = False

            # Initialize metrics for this phase
            running_loss = 0.0  # Accumulate losses over the epoch
            running_iou = 0.0
            correct = 0
            with tqdm(total=len(dataloaders[phase]), unit='batch') as p:

                for batch_idx, (inputs, labels, weights, _) in enumerate(dataloaders[phase]):
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    weights = weights.to(device)
                    optimizer.zero_grad()

                    batch_weights = torch.mean(weights, dim=0)
                    with torch.set_grad_enabled(is_training):
                        with torch.autocast(device_type=device, dtype=torch.float16):
                            outputs = model(inputs).squeeze()
                            loss = F.cross_entropy(outputs, labels.squeeze(), ignore_index=-1, weight=batch_weights, label_smoothing=0.05)
                            if is_training:
                                if device=="cuda":
                                    scaler.scale(loss).backward()
                                    scaler.step(optimizer)
                                    scaler.update()
                                else:
                                    loss.backward()
                                    optimizer.step()
                                scheduler.step()

                    # Update running loss and correct prediction count
                    running_loss += loss.item()

                    with torch.no_grad():
                        mIoU = jaccard(outputs, labels.squeeze())#mean_iou.compute(predictions=preds, references=labels.squeeze(), num_labels=3, ignore_index=255)
                        running_iou += mIoU

                    # Update the progress bar
                    p.set_postfix({'loss': running_loss/(batch_idx+1),  "mIoU" : running_iou/(batch_idx+1)})
                    p.update(1)

                # Calculate loss, accuracy, and Jaccard Index for this epoch
                epoch_loss = running_loss /(batch_idx+1)
                epoch_iou = running_iou /(batch_idx+1)
                if not is_training:
                    if epoch_iou > best_val_iou:
                        best_val_iou = epoch_iou
                        torch.save(model.state_dict(), save_directory+"model_best_iou.pth")

                    if epoch_loss < best_val_loss:
                        best_val_loss = epoch_loss
                        torch.save(model.state_dict(), save_directory+"model_best_loss.pth")


                loss_dict[phase][epoch] = epoch_loss
                metrics_dict[phase]["mean_iou"].append(epoch_iou.cpu().item())

    # Get the maximum IoU value from the 'val' phase
    max_val_iou = max(metrics_dict['val']["mean_iou"])

    print(f"The maximum IoU value in the validation set is: {max_val_iou}")
    save_dicts(loss_dict, metrics_dict, save_directory)

    plot_learning_curves(loss_dict, metrics_dict, save_directory)

    return model

# Set the number of filters based on the size of the network
if size_network == "small":
    n_filters = 32
elif size_network == "large":
    n_filters = 64

# Calculate the number of training steps
num_training_steps = epoch_nb * len(dataloaders["train"])

# Initialize the model
model = UnetSegmentor(num_classes=num_classes, n_filter=n_filters)
model.to(device)

# Define the loss function
loss = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

# Define the learning rate scheduler
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=10, num_training_steps=num_training_steps)

print(f"Number of parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

# Train the model
model = train_segmentation_model(model, dataloaders, loss, optimizer, lr_scheduler, device, epoch_nb)

# Save the experiment parameters and model state
save_experiment_params(num_samples,
                       batch_size,
                       num_dataset,
                       epoch_nb,
                       lr,
                       weight_decay,
                       model)

# **Inference**

In [ ]:
# Initialize the model
model = UnetSegmentor(num_classes=num_classes)
model.to(device)

# Load state dictionary
model_name = "model_best_iou.pth"
checkpoint = torch.load(os.path.join(save_directory, model_name), map_location=torch.device('cpu'))

# Load the model's state dictionary
model.load_state_dict(checkpoint)

# Set the model to evaluation mode
model.eval()

Run this for a random prediction on the val set

In [ ]:
# Get a batch of validation data
data_iter = iter(dataloaders['val'])
data_batch = next(data_iter)

# Extract the first image and mask from the batch
image_presentation = data_batch[3][0]  # First image for presentation
image = data_batch[0][0].unsqueeze(0)  # First image with batch dimension added
mask = data_batch[1][0].unsqueeze(0)   # First mask with batch dimension added

# Move the model to the specified device (CPU or GPU)
model.to(device)

# Predict the mask using the model
with torch.no_grad():
    image = image.to(device)  # Move the image to the specified device
    predicted_mask = model(image)  # Get the predicted mask from the model
    preds = torch.argmax(predicted_mask, dim=1).cpu()  # Get the class with the highest probability for each pixel

def pred_function(image, mask, preds):
    """
    Displays the image, ground truth mask, and predicted mask.

    Args:
        image (torch.Tensor): The input image.
        mask (torch.Tensor): The ground truth mask.
        preds (torch.Tensor): The predicted mask.
    """
    # Print the shape and value range of the image and mask
    print(f"Image shape: {image.shape}")
    print(f"Mask shape: {mask.shape}")
    print(f"Minimum value of the image: {image.min().item()}")
    print(f"Maximum value of the image: {image.max().item()}")
    print(f"Minimum value of the mask: {mask.min().item()}")
    print(f"Maximum value of the mask: {mask.max().item()}")

    # Create subplots to display the image, mask, and predicted mask
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Plot the image
    image = image.numpy().transpose(1, 2, 0)  # Convert to numpy and transpose for correct display
    axes[0].imshow(image)
    axes[0].set_title('Image')

    # Plot the ground truth mask
    mask = mask[0].numpy()  # Convert to numpy
    axes[1].imshow(mask)
    axes[1].set_title('Mask')

    # Plot the predicted mask
    preds = preds.squeeze().numpy()  # Convert to numpy and remove extra dimensions
    axes[2].imshow(preds)
    axes[2].set_title('Predicted Mask')

    plt.show()

def plot_learning_curves_from_pt(loss_tensor_file, jaccard_tensor_file):
    """
    Plots the learning curves for loss and IoU from saved tensor files.

    Args:
        loss_tensor_file (str): Path to the file containing loss tensors.
        jaccard_tensor_file (str): Path to the file containing IoU tensors.
    """
    # Load tensors from files
    loss_tensor = torch.load(loss_tensor_file)
    jaccard_tensor = torch.load(jaccard_tensor_file)

    # Convert tensors to lists
    train_loss_values = loss_tensor['train'].tolist()
    val_loss_values = loss_tensor['val'].tolist()
    train_jaccard_values = jaccard_tensor['train'].tolist()
    val_jaccard_values = jaccard_tensor['val'].tolist()

    # Extract epochs
    epochs = list(range(1, len(train_loss_values) + 1))

    # Plot learning curves
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Plot loss values
    ax0 = axes[0]
    ax0.plot(epochs, train_loss_values, 'b-', label='train')
    ax0.plot(epochs, val_loss_values, 'r-', label='val')
    ax0.set_title('Loss')
    ax0.set_xlabel('Epochs')
    ax0.set_ylabel('Loss')

    # Plot IoU values
    ax2 = axes[1]
    ax2.plot(epochs, train_jaccard_values, 'b-', label='train')
    ax2.plot(epochs, val_jaccard_values, 'r-', label='val')
    ax2.set_title('IoU')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('%')
    ax2.legend()

    plt.tight_layout()
    plt.show()

# Load the saved tensors
loss_tensor = torch.load(os.path.join(save_directory, 'loss_tensor.pt'))
jaccard_tensor = torch.load(os.path.join(save_directory, 'iou_tensor.pt'))

# Find the largest IoU value for 'val' tensors
largest_iou_val = jaccard_tensor['val'].max().item()

# Find the smallest loss value for 'val' tensors
smallest_loss_val = loss_tensor['val'].min().item()

# Print the largest IoU and smallest loss values
print("Largest IoU value for 'val' tensors:", largest_iou_val)
print("Smallest loss value for 'val' tensors:", smallest_loss_val)


# Display the image, ground truth mask, and predicted mask
pred_function(image_presentation.cpu(), mask.cpu(), preds.cpu())


# Plot the learning curves
plot_learning_curves_from_pt(os.path.join(save_directory, 'loss_tensor.pt'),
                             os.path.join(save_directory, 'iou_tensor.pt'))